In [3]:
from moviepy.editor import VideoFileClip

def extract_audio_from_video(video_file, output_audio_file):
    video = VideoFileClip(video_file)
    audio = video.audio
    audio.write_audiofile(output_audio_file)

# Example usage
# extract_audio_from_video('path_to_video.mp4', 'output_audio.mp3')


from moviepy.editor import VideoFileClip, AudioFileClip

def merge_audio_with_video(video_file, audio_file, output_video_file):
    video = VideoFileClip(video_file)
    audio = AudioFileClip(audio_file)
    final_video = video.set_audio(audio)
    final_video.write_videofile(output_video_file, codec='libx264', audio_codec='aac')

# Example usage
# merge_audio_with_video('path_to_video.mp4', 'new_audio.mp3', 'output_video_with_new_audio.mp4')


In [19]:
from pydub import AudioSegment
import numpy as np

import numpy as np
from pydub import AudioSegment
import librosa

def generate_semitone_changes(audio_file, durations, semitone_values, smoothing='linear'):
    audio = AudioSegment.from_file(audio_file)
    result = AudioSegment.empty()

    def apply_semitone_change(segment, start_semitone, end_semitone, smoothing):
        # Convert AudioSegment to NumPy array
        segment_array = np.array(segment.get_array_of_samples())
        sample_rate = segment.frame_rate

        if smoothing == 'logarithmic':
            # Generate a logarithmic curve for semitone change
            x = np.linspace(1, np.e, len(segment_array))
            semitones = np.log(x) / np.log(x).max() * (end_semitone - start_semitone) + start_semitone
        elif smoothing == 'none':
            # Apply no smoothing, use end_semitone for the entire segment
            return librosa.effects.pitch_shift(segment_array, sample_rate, end_semitone)
        else:  # Linear smoothing (default)
            semitones = np.linspace(start_semitone, end_semitone, len(segment_array))

        modified_segment = []
        for i, semitone in enumerate(semitones):
            shifted = librosa.effects.pitch_shift(segment_array[i:i+1], sr=sample_rate, n_steps=semitone)
            modified_segment.extend(shifted)

        return AudioSegment(np.array(modified_segment).tobytes(), frame_rate=sample_rate, sample_width=segment.sample_width, channels=segment.channels)

    for (start, end), (start_semitone, end_semitone) in zip(durations, semitone_values):
        segment = audio[start*1000:end*1000]
        result += apply_semitone_change(segment, start_semitone, end_semitone, smoothing)

    return result



# Example usage
# durations = [(0, 13), (13, 24), (24, 34), (34, 40), (40, 68), (68, 90)]
# semitone_values = [(12, 12), (12, 5), (5, 12), (12, 12), (12, -6), (-6, 12)]
# modified_audio = generate_smooth_semitone_changes("path_to_audio_file.wav", durations, semitone_values, smoothing='logarithmic')


def save_audio_segment(audio_segment, output_file):
    """
    Save an audio segment to a file.

    :param audio_segment: The audio segment to save.
    :param output_file: The name of the output file.
    """
    # Export audio segment to a file
    audio_segment.export(output_file, format='mp3')


In [33]:
from pydub import AudioSegment
import numpy as np

def apply_semitone_change(segment, start_semitone, end_semitone, smoothing):
    # Convert AudioSegment to NumPy array
    segment_samples = np.array(segment.get_array_of_samples())
    
    # Normalize the array to floating-point values between -1.0 and 1.0
    dtype = segment_samples.dtype
    max_int_value = np.iinfo(dtype).max
    segment_samples = segment_samples.astype(np.float32) / max_int_value

    sample_rate = segment.frame_rate

    # Calculate average semitone change for the segment
    semitone_change = (start_semitone + end_semitone) / 2

    # Apply pitch shift to the entire segment
    shifted_samples = librosa.effects.pitch_shift(segment_samples, sr=sample_rate, n_steps=semitone_change)

    # Convert the floating-point array back to the original integer format
    shifted_samples = (shifted_samples * max_int_value).astype(dtype)

    return AudioSegment(shifted_samples.tobytes(), frame_rate=sample_rate, sample_width=segment.sample_width, channels=segment.channels)


def change_pitch_and_duration(segment, semitone_change):
    # Extract samples from segment
    samples = np.array(segment.get_array_of_samples())
    if segment.sample_width == 2:  # sample_width of 2 means 16-bit audio
        samples = samples.astype(np.int16)
    
    # Normalize the samples to -1.0 to 1.0 range for librosa
    normalized_samples = librosa.util.buf_to_float(samples, n_bytes=segment.sample_width)
    
    # Change the pitch with librosa
    pitched_samples = librosa.effects.pitch_shift(normalized_samples, sr=segment.frame_rate, n_steps=semitone_change)

    # Normalize pitched audio to prevent clipping
    max_int_value = float(int('1' * (segment.sample_width * 8), 2))  # Max value for the sample width
    pitched_samples = np.int16(pitched_samples / np.max(np.abs(pitched_samples)) * max_int_value / 2)
    
    # Create a new AudioSegment with the pitched samples
    new_segment = AudioSegment(
        pitched_samples.tobytes(),
        frame_rate=segment.frame_rate,
        sample_width=segment.sample_width,
        channels=segment.channels
    )
    
    return new_segment

def generate_smooth_semitone_changes(durations, semitone_values):
    """
    Generates a list of smoothly transitioning semitone changes for specified durations.
    
    :param durations: A list of tuples indicating the start and end times in seconds.
    :param semitone_values: A list of semitone values to transition between.
    :return: A list of semitone changes.
    """
    total_duration = sum(end - start for start, end in durations)
    frame_rate = 1000  # Assuming each frame is 1 second for simplicity
    total_frames = total_duration * 1000 / frame_rate
    semitone_changes = [0] * int(total_frames)
    
    for (start, end), (start_semitone, end_semitone) in zip(durations, semitone_values):
        start_frame = int(start * 1000 / frame_rate)
        end_frame = int(end * 1000 / frame_rate)
        for i in range(start_frame, end_frame):
            # Calculate the fraction of the way through the transition
            fraction = (i - start_frame) / (end_frame - start_frame)
            # Calculate the current semitone
            semitone_changes[i] = start_semitone + fraction * (end_semitone - start_semitone)
    
    return semitone_changes



def generate_semitone_changes(audio_file, durations, semitone_values, smoothing='linear'):
    audio = AudioSegment.from_file(audio_file)
    result = AudioSegment.empty()

    for (start, end), (start_semitone, end_semitone) in zip(durations, semitone_values):
        segment = audio[start*1000:end*1000]
        result += apply_semitone_change(segment, start_semitone, end_semitone, smoothing)

    return result


# Function to break a song into segments
def make_segments(song, frame_duration_ms):
    return [song[i:i+frame_duration_ms] for i in range(0, len(song), frame_duration_ms)]

def process_file(input_file, frame_duration_ms, semitone_changes):
    song = AudioSegment.from_file(input_file, format="mp3")
    segments = make_segments(song, frame_duration_ms)
    processed_segments = [change_pitch_and_duration(segment, semitone) for segment, semitone in zip(segments, semitone_changes)]
    return sum(processed_segments)




# Example usage
# durations = [(0, 10), (10, 20), (20, 30)]
# speed_changes = [(1.0, 0.5), (0.5, 2.0), (2.0, 1.0)]
# modified_audio = change_audio_speed("path_to_audio_file.wav", durations, speed_changes, smoothing='logarithmic')


In [6]:
import numpy as np
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips
from pydub import AudioSegment

def apply_speed_change(clip, start, end, start_speed, end_speed, smoothing):
    duration = end - start
    if smoothing == 'logarithmic':
        # Generate a logarithmic curve for speed change
        x = np.linspace(1, np.e, duration)
        speeds = np.log(x) / np.log(x).max() * (end_speed - start_speed) + start_speed
    else:  # Linear smoothing
        speeds = np.linspace(start_speed, end_speed, duration)

    modified_clip = []
    for i, speed in enumerate(speeds):
        segment = clip.subclip(start + i, start + i + 1).speedx(speed)
        modified_clip.append(segment)

    return concatenate_videoclips(modified_clip) if isinstance(clip, VideoFileClip) else sum(modified_clip)

def modify_speed(audio_file, video_file, durations, speed_changes, smoothing='linear'):
    audio_clip = AudioFileClip(audio_file)
    video_clip = VideoFileClip(video_file)

    modified_audio = AudioSegment.empty()
    modified_video = []

    for (start, end), (start_speed, end_speed) in zip(durations, speed_changes):
        audio_segment = audio_clip.subclip(start, end)
        video_segment = video_clip.subclip(start, end)

        modified_audio += apply_speed_change(audio_segment, start, end, start_speed, end_speed, smoothing)
        modified_video.append(apply_speed_change(video_segment, start, end, start_speed, end_speed, smoothing))

    final_audio = concatenate_audioclips(modified_audio)
    final_video = concatenate_videoclips(modified_video)

    final_video.set_audio(final_audio)
    return final_video

# Example usage
# durations = [(0, 10), (10, 20), (20, 30)]
# speed_changes = [(1.0, 0.5), (0.5, 2.0), (2.0, 1.0)]
# modified_video = modify_speed("path_to_audio.wav", "path_to_video.mp4", durations, speed_changes, smoothing='logarithmic')
# modified_video.write_videofile("modified_video.mp4")


In [8]:
import re
def get_duration_in_seconds(audio_file):
    audio = AudioSegment.from_file(audio_file)
    return len(audio) / 1000


def time_to_seconds(time_str):
    """Converts a time string MM:SS to seconds."""
    minutes, seconds = map(int, time_str.split(":"))
    return minutes * 60 + seconds


def parse_string(s):
    # Regex pattern to extract the necessary components
    pattern = r'(\d{2}:\d{2})\s*->\s*(\d{2}:\d{2})\s*(-?\d+)(?:\s*->\s*(-?\d+))?'
    
    # Search for the pattern in the string
    match = re.search(pattern, s)

    if match:
        # Extract captured groups
        start_time, end_time, semitone1, semitone2 = match.groups()
        
        # If the second semitone is not present, use the first one again
        semitone2 = semitone2 if semitone2 is not None else semitone1
        
        return time_to_seconds(start_time), time_to_seconds(end_time), float(semitone1), float(semitone2)
    else:
        return None


def get_durations_and_values(s):
    durs = s.split('\n')
    durations = []
    semitone_values = []

    for dur in durs:
        d0,d1, s0,s1 = parse_string(dur)
        durations.append((d0,d1))
        semitone_values.append((s0,s1))
        
    return durations, semitone_values

In [36]:
file_name = 'Kardeş Türküler - Bugün Güzellerin Şahını Gördüm & Dem Ali\'ye [ Hemâvâz © 2011 Kalan Müzik ]'
project_name = 'p1'
# extract_audio_from_video(f'./musics/{project_name}/{file_name}.mp4', f'./musics/{project_name}/{file_name}.mp3') step1

s = '''00:00 -> 00:36  5
    00:36 -> 00:46  5 -> -2
    00:46 -> 01:02  -2'''
    # 00:25 -> 00:32  -12
    # 00:32 -> 00:39  -12 -> 2
    # 00:39 -> 01:07  2
    # 01:02 -> 01:08  -5 -> -12
    # 01:08 -> 01:14  -12 
    # 01:14 -> 01:27  -12 -> -5
    # 01:27 -> 01:32  -5
    # 01:32 -> 01:35  -5 -> 2
    # 01:35 -> 01:54  2
    # 01:54 -> 02:00  2 -> -5
    # 02:00 -> 02:04  -5
    # 02:04 -> 02:12  -5 -> 5
    # 02:12 -> 04:13  8
    
durations, semitone_values = get_durations_and_values(s)
speed_changes = calculate_speed_changes_for_pitch(durations, semitone_values)

# semitone_changes_audio = generate_semitone_changes(f'./musics/{project_name}/{file_name}.mp3', durations, semitone_values)
# save_audio_segment(semitone_changes_audio, f'./musics/{project_name}/{file_name}-semitone.mp3')

frame_duration_ms = 1000
processed_song = process_file(f'./musics/{project_name}/{file_name}.mp3', frame_duration_ms, generate_smooth_semitone_changes(durations, semitone_values))
processed_song.export(f'./musics/{project_name}/{file_name}-semitone.mp3', format="mp3")


# merge_audio_with_video('path_to_video.mp4', 'new_audio.mp3', 'output_video_with_new_audio.mp4')


<_io.BufferedRandom name="./musics/p1/Kardeş Türküler - Bugün Güzellerin Şahını Gördüm & Dem Ali'ye [ Hemâvâz © 2011 Kalan Müzik ]-semitone.mp3">

In [ ]:
import math
import random
# Function to generate semitone changes based on a ReLU activation-style function
def generate_relu_semitones(total_frames, max_semitone_change):
    return [max(0, i - total_frames // 2) / (total_frames // 2) * max_semitone_change for i in range(total_frames)]

# Function to generate semitone changes with a sine wave pattern
def generate_sine_semitones(total_frames, max_semitone_change):
    return [math.sin(math.pi * i / total_frames) * max_semitone_change for i in range(total_frames)]

# Function to generate semitone changes with a cosine wave pattern
def generate_cosine_semitones(total_frames, max_semitone_change):
    return [math.cos(math.pi * i / total_frames) * max_semitone_change for i in range(total_frames)]

# Function to generate semitone changes with a random walk pattern
def generate_random_walk_semitones(total_frames, max_semitone_change, step=1):
    current_tone = 0
    semitones = [0]  # Start with no change
    for _ in range(1, total_frames):
        change = random.choice([-step, 0, step])
        current_tone += change
        current_tone = max(-max_semitone_change, min(max_semitone_change, current_tone))
        semitones.append(current_tone)
    return semitones


In [6]:
from pydub import AudioSegment

# Function to break a song into segments
def make_segments(song, frame_duration_ms):
    return [song[i:i+frame_duration_ms] for i in range(0, len(song), frame_duration_ms)]

# Function to change the pitch of a segment
def change_pitch(segment, semitone_change):
    octaves = semitone_change / 12
    new_sample_rate = int(segment.frame_rate * (2.0 ** octaves))
    return segment._spawn(segment.raw_data, overrides={'frame_rate': new_sample_rate}).set_frame_rate(segment.frame_rate)

# Function to change the speed of a segment
def change_speed(segment, speed_factor):
    return segment._spawn(segment.raw_data, overrides={'frame_rate': int(segment.frame_rate * speed_factor)}).set_frame_rate(segment.frame_rate)



def generate_smooth_semitone_changes(durations, semitone_values):
    """
    Generates a list of smoothly transitioning semitone changes for specified durations.
    
    :param durations: A list of tuples indicating the start and end times in seconds.
    :param semitone_values: A list of semitone values to transition between.
    :return: A list of semitone changes.
    """
    total_duration = sum(end - start for start, end in durations)
    frame_rate = 1000  
    total_frames = total_duration * 1000 / frame_rate
    semitone_changes = [0] * int(total_frames)
    
    for (start, end), (start_semitone, end_semitone) in zip(durations, semitone_values):
        start_frame = int(start * 1000 / frame_rate)
        end_frame = int(end * 1000 / frame_rate)
        for i in range(start_frame, end_frame):
            # Calculate the fraction of the way through the transition
            fraction = (i - start_frame) / (end_frame - start_frame)
            # Calculate the current semitone
            semitone_changes[i] = start_semitone + fraction * (end_semitone - start_semitone)
    
    return semitone_changes

# Main function to process the file with pitch changes and potentially other effects
def process_file(input_file, frame_duration_ms, semitone_changes):
    song = AudioSegment.from_file(input_file, format="mp3")
    segments = make_segments(song, frame_duration_ms)
    processed_segments = [change_pitch(segment, semitone) for segment, semitone in zip(segments, semitone_changes)]
    return sum(processed_segments)



from utils import get_durations_and_values 
from glob import glob
import os

project_name = 'p1'
file_name = os.path.basename(sorted(glob(f'./musics/{project_name}/*'), key=len)[0])[:-4]

frame_duration_ms = 1000  # 1 second frames

s = '''00:00 -> 00:36  5.0
00:36 -> 00:46  5.0 -> -2.0
00:46 -> 01:02  -2.0'''


durations, semitone_values = get_durations_and_values(s)
semitone_changes = generate_smooth_semitone_changes(durations, semitone_values)
processed_song = process_file(f'./musics/{project_name}/{file_name}.mp3', frame_duration_ms, semitone_changes)
processed_song.export(f'./musics/{project_name}/{file_name}-semitone.mp3', format="mp3")


<_io.BufferedRandom name="./musics/p1/Kardeş Türküler - Bugün Güzellerin Şahını Gördüm & Dem Ali'ye [ Hemâvâz © 2011 Kalan Müzik ]-semitone.mp3">

In [19]:
import librosa
import numpy as np
from pydub import AudioSegment

def change_speed(segment, speed_factor):
    # Convert Pydub AudioSegment to NumPy array
    samples = np.array(segment.get_array_of_samples())
    sample_rate = segment.frame_rate

    # Normalize the audio samples for librosa processing
    samples = samples.astype(np.float32) / float(2**15)

    # Handle stereo audio
    if segment.channels == 2:
        samples = samples.reshape((-1, 2))
        samples_left = samples[:, 0]
        samples_right = samples[:, 1]

        # Time-stretch each channel using the new librosa syntax
        stretched_left = librosa.effects.time_stretch(samples_left, rate=speed_factor)
        stretched_right = librosa.effects.time_stretch(samples_right, rate=speed_factor)

        # Combine the channels back
        stretched = np.vstack((stretched_left, stretched_right)).T
    else:
        # Time-stretch for mono audio
        stretched = librosa.effects.time_stretch(samples, rate=speed_factor)

    # Convert back to int16 and then to bytes for Pydub
    stretched = (stretched * float(2**15)).astype(np.int16)
    new_segment = AudioSegment(
        stretched.tobytes(),
        frame_rate=sample_rate,
        sample_width=segment.sample_width,
        channels=segment.channels
    )

    return new_segment

def generate_smooth_speed_changes(durations, speed_values):
    """
    Generates a list of smoothly transitioning speed changes for specified durations.
    
    :param durations: A list of tuples indicating the start and end times in seconds.
    :param speed_values: A list of speed values to transition between.
    :return: A list of speed changes.
    """
    total_duration = sum(end - start for start, end in durations)
    frame_rate = 1000  # Milliseconds in a second
    total_frames = total_duration * 1000 / frame_rate
    speed_changes = [1.0] * int(total_frames)  # Default speed factor is 1.0 (no change)
    
    for (start, end), (start_speed, end_speed) in zip(durations, speed_values):
        start_frame = int(start * 1000 / frame_rate)
        end_frame = int(end * 1000 / frame_rate)
        for i in range(start_frame, end_frame):
            # Calculate the fraction of the way through the transition
            fraction = (i - start_frame) / (end_frame - start_frame)
            # Calculate the current speed
            speed_changes[i] = start_speed + fraction * (end_speed - start_speed)
    
    return speed_changes

def process_file(input_file, frame_duration_ms, speed_changes):
    song = AudioSegment.from_file(input_file, format="mp3")
    segments = make_segments(song, frame_duration_ms)
    processed_segments = [change_speed(segment, speed) for segment, speed in zip(segments, speed_changes)]
    return sum(processed_segments)

# Example usage
speed_change_str = '''00:00 -> 00:19  0.5
00:19 -> 00:27  0.5 -> 2.5 
00:27 -> 00:54  1.0'''

# You need to implement get_durations_and_speeds to parse the string
durations, speed_values = get_durations_and_values(speed_change_str)
speed_changes = generate_smooth_speed_changes(durations, speed_values)


processed_durations = []
for semitone_change in semitone_changes:
    speed_change_factor = 2 ** (semitone_change / 12)
    new_duration = 1 / speed_change_factor
    processed_durations.append(new_duration)
# processed_durations = speed_changes


processed_song = process_file(f'./musics/{project_name}/{file_name}-semitone.mp3', frame_duration_ms, processed_durations)
processed_song.export(f'./musics/{project_name}/{file_name}-speed.mp3', format="mp3")


<_io.BufferedRandom name="./musics/p1/Kardeş Türküler - Bugün Güzellerin Şahını Gördüm & Dem Ali'ye [ Hemâvâz © 2011 Kalan Müzik ]-speed.mp3">